In [ ]:
import dotenv
dotenv.load_dotenv()

## 빗썸 티커 리스트 가져오는 함수 정의

In [1]:
import requests
import json

In [2]:
def get_bithumb_krw_ticker_list():
    url = "https://api.bithumb.com/v1/market/all?is_Details=false"
    headers = {"accept" : "application/json"}
    response = requests.get(url, headers=headers)

    # JSON 문자열을 파싱하여 리스트로 변환
    parsed_data = json.loads(response.text)
    # KRW-로 시작하는 항목만 추출
    krw_markets = [item for item in parsed_data if item["market"].startswith("KRW-")]
    # market 값만 추출
    market_list = [item["market"] for item in krw_markets]

    return market_list

In [4]:
get_bithumb_krw_ticker_list()[:10]

['KRW-BTC',
 'KRW-ETH',
 'KRW-ETC',
 'KRW-XRP',
 'KRW-BCH',
 'KRW-QTUM',
 'KRW-A',
 'KRW-ICX',
 'KRW-TRX',
 'KRW-ELF']

## 특정 날짜의 빗썸 리더보드 순위를 재현하는 함수 정의

In [6]:
import time
from tqdm import tqdm
from datetime import datetime, timedelta

In [7]:
def get_bithumb_candle_data(market: str, to: str, count: int = 1):
    """
    Bithumb API에서 특정 시장과 날짜의 캔들 데이터를 가져오는 함수.

    Args:
        market (str): 조회할 마켓 (예: "KRW-MVC").
        to (str): 특정 날짜의 마지막 캔들 시각 (ISO8061 포맷, 예: "2024-12-15T00:00:00").
        count (int, optional): 가져올 데이터 개수. 기본값은 1.

    Returns:
        dict: API 응답 데이터 (JSON 형식).
    """
    # API URL
    url = "https://api.bithumb.com/v1/candles/days"

    # 파라미터 설정
    params = {
        "count": count,
        "to": to,
        "market": market
    }

    try:
        # GET 요청
        response = requests.get(url, params=params)
        response.raise_for_status()  # HTTP 에러 발생 시 예외 처리

        # JSON 데이터 반환
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error occurred: {e}")
        return None

In [8]:
def calculate_high_open_ratio_and_volume(result):
    """
    시가 대비 고가 상승률과 거래 대금을 억 단위로 계산하는 함수.

    Args:
        result (list): API 응답 데이터 (리스트 형식, 각 딕셔너리 내 시가, 고가, 거래 대금 포함).

    Returns:
        tuple: 시가 대비 고가 상승률(%)과 거래 대금(억 단위)을 반환.
    """
    if not result or "opening_price" not in result[0] or "high_price" not in result[0] or "candle_acc_trade_price" not in result[0]:
        raise ValueError("유효한 데이터 형식이 아닙니다.")

    # 시가, 고가, 거래 대금 가져오기
    opening_price = result[0]["opening_price"]
    high_price = result[0]["high_price"]
    trade_volume = result[0]["candle_acc_trade_price"]  # 거래 대금

    # 상승률 계산
    high_open_ratio = ((high_price - opening_price) / opening_price) * 100

    # 거래 대금 억 단위로 변환
    trade_volume_in_100_million = trade_volume / 1e8

    return high_open_ratio, trade_volume_in_100_million

In [10]:
from langchain_core.tools import tool

@tool
def get_bithumb_daily_increase_rate_information(date_string : str) -> str:
    """
    주어진 날짜를 기준으로 빗썸의 모든 KRW 마켓의 시가 대비 고가 상승률과 거래 대금 정보를 계산하여 반환하는 함수.

    Parameters:
        date_string (str): 분석하고자 하는 기준 날짜. 형식은 "YYYY-MM-DD"이어야 합니다.

    Returns:
        str: 각 마켓의 시가 대비 고가 상승률과 거래 대금을 포함한 정렬된 결과 문자열.
             각 행은 "마켓명 시가 대비 고가 상승률: X.XX%, 거래 대금: Y.YY억 원" 형식으로 구성됩니다.

    주요 기능:
        1. 입력된 날짜 문자열을 기준으로 다음 날 자정(00:00:00)의 시간 문자열 생성.
        2. 빗썸의 KRW 마켓 리스트를 가져와 각 마켓의 데이터를 처리.
        3. 각 마켓의 시가 대비 고가 상승률 및 거래 대금 계산:
            - `calculate_high_open_ratio_and_volume`를 사용하여 상승률과 거래 대금 도출.
        4. 상승률(high_open_ratio)을 기준으로 결과 정렬(내림차순).
        5. 최종적으로 각 마켓의 결과를 문자열로 포맷하여 반환.

    참고:
        - `get_bithumb_krw_ticker_list`: 빗썸의 KRW 마켓 리스트를 가져오는 함수.
        - `get_bithumb_candle_data`: 특정 마켓과 날짜에 해당하는 캔들 데이터를 가져오는 함수.
        - `calculate_high_open_ratio_and_volume`: 상승률 및 거래 대금을 계산하는 함수.
        - `time.sleep(0.01)`: API 호출 간격 조정.
        - 결과는 시가 대비 고가 상승률 기준으로 내림차순 정렬됩니다.

    예외 처리:
        - 데이터가 유효하지 않을 경우 해당 마켓은 건너뜀.
        - `calculate_high_open_ratio_and_volume` 처리 중 오류 발생 시 해당 마켓 건너뜀.

    """

    market_list = get_bithumb_krw_ticker_list()

    current_date = datetime.strptime(date_string, "%Y-%m-%d")

    # 결과 저장용 리스트
    results = []

    """
    날짜 문자열 생성
    """
    next_date = current_date + timedelta(days=1)  # 다음 날짜 계산
    date_str = next_date.strftime("%Y-%m-%dT00:00:00")  # 다음 날 00:00:00 설정

    """
    시장 데이터 처리 루프
    """
    for market in market_list:
        to = date_str  # 특정 날짜의 마지막 캔들

        """
        데이터 가져오기
        """
        result = get_bithumb_candle_data(market, to)

        if result:  # 데이터가 유효한 경우 처리
            try:
                """
                상승률과 거래 대금 계산
                """
                ratio, volume = calculate_high_open_ratio_and_volume(result)

                """
                결과 저장
                """
                results.append({
                    "market": market,
                    "high_open_ratio": ratio,
                    "trade_volume": volume
                })
            except ValueError as e:
                print(f"{market} 처리 중 에러: {e}")

        """
        요청 간격 조정
        """
        time.sleep(0.01)

    current_date += timedelta(days=1)  # 다음 날짜로 이동

    """
    시가 대비 고가 상승률 기준으로 결과 정렬
    """
    sorted_results = sorted(results, key=lambda x: x["high_open_ratio"], reverse=True)

    """
    정렬된 결과 출력
    """
    output = "\n".join(
        f"{item['market']} 시가 대비 고가 상승률: {item['high_open_ratio']:.2f}%, 거래 대금: {item['trade_volume']:.2f}억 원"
        for item in sorted_results
    )

    return output

In [11]:
print(get_bithumb_daily_increase_rate_information.name)
print(get_bithumb_daily_increase_rate_information.description)
print(get_bithumb_daily_increase_rate_information.args)

get_bithumb_daily_increase_rate_information
주어진 날짜를 기준으로 빗썸의 모든 KRW 마켓의 시가 대비 고가 상승률과 거래 대금 정보를 계산하여 반환하는 함수.

Parameters:
    date_string (str): 분석하고자 하는 기준 날짜. 형식은 "YYYY-MM-DD"이어야 합니다.

Returns:
    str: 각 마켓의 시가 대비 고가 상승률과 거래 대금을 포함한 정렬된 결과 문자열.
         각 행은 "마켓명 시가 대비 고가 상승률: X.XX%, 거래 대금: Y.YY억 원" 형식으로 구성됩니다.

주요 기능:
    1. 입력된 날짜 문자열을 기준으로 다음 날 자정(00:00:00)의 시간 문자열 생성.
    2. 빗썸의 KRW 마켓 리스트를 가져와 각 마켓의 데이터를 처리.
    3. 각 마켓의 시가 대비 고가 상승률 및 거래 대금 계산:
        - `calculate_high_open_ratio_and_volume`를 사용하여 상승률과 거래 대금 도출.
    4. 상승률(high_open_ratio)을 기준으로 결과 정렬(내림차순).
    5. 최종적으로 각 마켓의 결과를 문자열로 포맷하여 반환.

참고:
    - `get_bithumb_krw_ticker_list`: 빗썸의 KRW 마켓 리스트를 가져오는 함수.
    - `get_bithumb_candle_data`: 특정 마켓과 날짜에 해당하는 캔들 데이터를 가져오는 함수.
    - `calculate_high_open_ratio_and_volume`: 상승률 및 거래 대금을 계산하는 함수.
    - `time.sleep(0.01)`: API 호출 간격 조정.
    - 결과는 시가 대비 고가 상승률 기준으로 내림차순 정렬됩니다.

예외 처리:
    - 데이터가 유효하지 않을 경우 해당 마켓은 건너뜀.
    - `calculate_high_open_ratio_and_volume` 처리 중 오류 발생 시 해당 마켓 

In [12]:
#  데이터 가져오기
coin_data = get_bithumb_daily_increase_rate_information.invoke({"date_string":'2025-01-01'})

# 출력
print(coin_data)

KRW-CVC 시가 대비 고가 상승률: 21.36%, 거래 대금: 270.72억 원
KRW-GOAT 시가 대비 고가 상승률: 20.59%, 거래 대금: 210.95억 원
KRW-AERGO 시가 대비 고가 상승률: 17.11%, 거래 대금: 126.98억 원
KRW-STEEM 시가 대비 고가 상승률: 16.88%, 거래 대금: 225.22억 원
KRW-GLM 시가 대비 고가 상승률: 16.33%, 거래 대금: 81.75억 원
KRW-TEMCO 시가 대비 고가 상승률: 16.33%, 거래 대금: 49.45억 원
KRW-VIRTUAL 시가 대비 고가 상승률: 15.24%, 거래 대금: 304.68억 원
KRW-MIX 시가 대비 고가 상승률: 14.54%, 거래 대금: 16.20억 원
KRW-NFT 시가 대비 고가 상승률: 14.29%, 거래 대금: 0.56억 원
KRW-XLM 시가 대비 고가 상승률: 13.02%, 거래 대금: 277.28억 원
KRW-ACS 시가 대비 고가 상승률: 12.33%, 거래 대금: 22.35억 원
KRW-WOM 시가 대비 고가 상승률: 11.48%, 거래 대금: 56.45억 원
KRW-LM 시가 대비 고가 상승률: 11.13%, 거래 대금: 27.10억 원
KRW-PENGU 시가 대비 고가 상승률: 11.09%, 거래 대금: 238.80억 원
KRW-GAS 시가 대비 고가 상승률: 10.21%, 거래 대금: 3.75억 원
KRW-ROA 시가 대비 고가 상승률: 9.98%, 거래 대금: 34.80억 원
KRW-SOFI 시가 대비 고가 상승률: 9.72%, 거래 대금: 6.47억 원
KRW-TDROP 시가 대비 고가 상승률: 9.70%, 거래 대금: 12.31억 원
KRW-HUNT 시가 대비 고가 상승률: 9.56%, 거래 대금: 13.56억 원
KRW-EGG 시가 대비 고가 상승률: 9.42%, 거래 대금: 9.32억 원
KRW-OBSR 시가 대비 고가 상승률: 8.89%, 거래 대금: 10.63억 원
KRW-CTC 시가 대비 고가 상승률

## Crypto Research Agent 생성

In [16]:
tools = [get_bithumb_daily_increase_rate_information]

In [17]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
)

In [18]:
llm_with_tools = llm.bind_tools(tools)

In [19]:
from langchain import hub

# Fetches the latest version of this prompt
prompt = hub.pull("wfh/langsmith-agent-prompt:5d466cbc")
prompt.pretty_print()

================================ System Message ================================

You are a helpful assistant.

================================ Human Message =================================

{input}

============================= Messages Placeholder =============================

{agent_scratchpad}


In [20]:
from langchain.agents import AgentExecutor
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

runnable_agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

agent_executor = AgentExecutor(
    agent=runnable_agent, tools=tools, handle_parsing_errors=True
)

In [21]:
results = agent_executor.invoke({"input":"2025년 1월 1일 빗썸에서 가장 많이 상승한 종목과 거래대금을 알려줘"})
results

{'input': '2025년 1월 1일 빗썸에서 가장 많이 상승한 종목과 거래대금을 알려줘',
 'output': '2025년 1월 1일 빗썸에서 가장 많이 상승한 종목은 **KRW-CVC**로, 시가 대비 고가 상승률이 21.36%였습니다. 해당 종목의 거래 대금은 270.72억 원이었습니다.'}

In [22]:
results = agent_executor.invoke({"input":"2024년 12월 14일 빗썸에서 가장 많이 상승한 종목 10개와 거래대금을 알려줘"})
results

{'input': '2024년 12월 14일 빗썸에서 가장 많이 상승한 종목 10개와 거래대금을 알려줘',
 'output': '2024년 12월 14일 빗썸에서 가장 많이 상승한 종목 10개와 그 거래대금은 다음과 같습니다:\n\n1. **KRW-MVC**: 시가 대비 고가 상승률 132.60%, 거래 대금 1095.13억 원\n2. **KRW-WIKEN**: 시가 대비 고가 상승률 80.42%, 거래 대금 646.13억 원\n3. **KRW-BLY**: 시가 대비 고가 상승률 79.65%, 거래 대금 305.09억 원\n4. **KRW-EL**: 시가 대비 고가 상승률 60.06%, 거래 대금 2316.34억 원\n5. **KRW-FLZ**: 시가 대비 고가 상승률 50.74%, 거래 대금 135.46억 원\n6. **KRW-MIX**: 시가 대비 고가 상승률 44.46%, 거래 대금 109.79억 원\n7. **KRW-APM**: 시가 대비 고가 상승률 43.16%, 거래 대금 115.87억 원\n8. **KRW-REQ**: 시가 대비 고가 상승률 33.68%, 거래 대금 100.45억 원\n9. **KRW-LM**: 시가 대비 고가 상승률 32.20%, 거래 대금 256.20억 원\n10. **KRW-MORPHO**: 시가 대비 고가 상승률 24.56%, 거래 대금 203.37억 원\n\n이 종목들은 시가 대비 고가 상승률을 기준으로 정렬된 결과입니다.'}